#  Rastrigin function optimization (max).
---
Description:

- Optimization (max)
- Single-objective
- Constraints (no)
---

The general equation is given by:

$f(\mathbf{x}) = An + \sum_{i=1}^{n} [x_i^2 - A\cos(2 \pi x_i)]$, with  $-5.12 \le x_i \le +5.12$, and $A = 10$.

According to [Rastrigin function](https://en.wikipedia.org/wiki/Rastrigin_function) for a given number of dimensions $n$ the maximum can be found accordingly.

To test here the code we set $n=5$, which gives the maximum of $201.7664509$ at coordinates $x_i \in [\pm 4.52299366, ..., \pm 4.52299366]$.

### First we import python libraries and set up the directory of our code.

In [3]:
import os, sys
import numpy as np
from math import fsum

PROJECT_DIR = os.path.abspath('..')
sys.path.append(PROJECT_DIR)

### Here we import all our custom GA code.

In [5]:
# Import main classes.
from pygenalgo.genome.gene import Gene
from pygenalgo.genome.chromosome import Chromosome
from pygenalgo.engines.standard_ga import StandardGA

# Import Selection Operator(s).
from pygenalgo.operators.selection.linear_rank_selector import LinearRankSelector

# Import Crossover Operator(s).
from pygenalgo.operators.crossover.meta_crossover import MetaCrossover

# Import Mutation Operator(s).
from pygenalgo.operators.mutation.meta_mutator import MetaMutator

### Define the Rastrigin function, which plays also the role of the 'fitness' function.

In addition, we define the 'rand_fx' which takes the role of the 'random()' method of the Genes. Every time we
want to 'mutate' a gene this function will be called and assign a 'valid', but random value to the gene.

In [7]:
# Rastrigin function.
def fun_Rastrigin(individual: Chromosome, f_min: bool = False):
    
    # Setup the model parameters.
    A, n = 10.0, len(individual)
    
    # Function value.
    f_val = A*n + fsum([xi.value**2 - A*np.cos(2.0*np.pi*xi.value) for xi in individual])
    
    # Return the f_val.
    return -f_val if f_min else f_val
# _end_def_

# Random function ~U(-5.12, +5.12).
rand_fx = lambda: np.random.uniform(-5.12, +5.12)

Here we set the GA parameters, such as number of genes, number of chromosomes, etc. Note that in this case each
gene has the same random() function (set by 'rand_fx'). But if the problem demands otherwise it is easy to set a 
different random() function for each gene.

In [9]:
# Define the number of genes.
M = 5

# Define the number of chromosomes.
N = 100

# Initial population.
population = [Chromosome([Gene(np.random.uniform(-5.0, +5.0), rand_fx)
                          for _ in range(M)], np.nan, True)
              for _ in range(N)]

# Create the StandardGA object that will carry on the optimization.
test_GA = StandardGA(initial_pop=population,
                     fit_func=fun_Rastrigin,
                     select_op=LinearRankSelector(),
                     mutate_op=MetaMutator(),
                     crossx_op=MetaCrossover())

### Optimization process.

Here we call the GA object (either directly, or through the method run()). We set a number of parameter,
such as the maximum iterations (i.e. epochs), tolerance for the fitness convergences, etc.

In [11]:
test_GA(epochs=1000, elitism=True, f_tol=1.0e-6, verbose=False)

Initial Avg. Fitness = 92.1638.
StandardGA finished in 106 iterations.
Final   Avg. Fitness = 201.7664.
Elapsed time: 0.853 seconds.


In [12]:
# Extract the optimal solution from the GA.
optimal_solution = test_GA.best_chromosome()

# Display the (final) optimum value.
print(f"Maximum Found: {fun_Rastrigin(optimal_solution, f_min=False):.5f}\n")

# Display each gene value separately.
for i, xi in enumerate(optimal_solution.genome):
    print(f"x{i} = {xi.value:>10.6f}")
# _end_for_

# True maximum: 201.7664509.

Maximum Found: 201.76642

x0 =  -4.523174
x1 =  -4.523174
x2 =  -4.523174
x3 =  -4.523174
x4 =  -4.523174


**Note that the above solution is indeed very close to the absolute maximum!**

In [14]:
# If we want we can also print some operator statistics.
test_GA.print_operator_stats()

 LinearRankSelector: (140521284410464)
 _probability: 1.0
 _counter: 106
 _lock: <unlocked _thread.lock object at 0x7fcda9589000>
 _items: None
 _iteration: 105

 MetaCrossover: (140521285792096)
 _probability: 0.9
 _counter: 4782
 _lock: <unlocked _thread.lock object at 0x7fcda9571400>
 _items: (UniformCrossover(1.0), MultiPointCrossover(1.0), SinglePointCrossover(1.0))
 _iteration: 0

 UniformCrossover: (140521285792288)
 _probability: 1.0
 _counter: 1648
 _lock: <unlocked _thread.lock object at 0x7fcda93ed580>
 _items: None
 _iteration: 0

 MultiPointCrossover: (140521285792192)
 _probability: 1.0
 _counter: 1531
 _lock: <unlocked _thread.lock object at 0x7fcda808ba40>
 _items: 2
 _iteration: 0

 SinglePointCrossover: (140521285792384)
 _probability: 1.0
 _counter: 1603
 _lock: <unlocked _thread.lock object at 0x7fcda92e2240>
 _items: None
 _iteration: 0

 MetaMutator: (140521285790752)
 _probability: 0.1
 _counter: 1047
 _lock: <unlocked _thread.lock object at 0x7fcda905bb40>
 _ite

### End of file